In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

# june_df = "data/Ebay_june2021_transactionreport.csv"
# july_df = "data/Ebay_july2021_transactionreport.csv"
# august_df = "data/Ebay_august2021_transactionreport.csv"
# sept_df = "data/Ebay_september2021_tranactionreport.csv"
# oct_df = "data/Ebay_october2021_transactionreport.csv"
# nov_df = "data/Ebay_nov2021_transactionreport.csv"
dec_df = "data/Ebay-Dec-07-2022-12-21.csv"

# add "data" to beginning of the path
df = pd.read_csv(dec_df, header=None, skiprows=11, skipfooter=0)
# Make row1 = column titles
df.columns = df.iloc[0]
df = df[1:]
df = df.iloc[::-1]
df.reset_index(inplace=True)
df.drop(['index'], inplace=True, axis=1)

df['Date'] = pd.to_datetime(df['Transaction creation date'])
df['Transaction creation date'] = df['Date']
df.drop(['Date'],inplace=True, axis=1)
df.rename({"Transaction creation date" : "date"}, inplace=True, axis=1)

df['Description'] = df['Description'].str.replace('--', 'Not Ad Fee')

df_dropped = df[['date', 'Net amount', 'Order number','Buyer name', 'Item title', 'Description',
                 'Item subtotal', 'eBay collected tax', 'Shipping and handling', 'Final Value Fee - fixed', 
                 'Final Value Fee - variable', 'Gross transaction amount']].copy()

df_dropped['Net amount'] = df_dropped['Net amount'].replace('--', 0)
df_dropped['Net amount'] = df_dropped['Net amount'].fillna(0)
df_dropped['Net amount'] = df_dropped['Net amount'].astype(float)

df_dropped['Item subtotal'] = df_dropped['Item subtotal'].replace('--', 0)
df_dropped['Item subtotal'] = df_dropped['Item subtotal'].fillna(0)
df_dropped['Item subtotal'] = df_dropped['Item subtotal'].astype(float)

df_dropped['Shipping and handling'] = df_dropped['Shipping and handling'].replace('--', 0)
df_dropped['Shipping and handling'] = df_dropped['Shipping and handling'].fillna(0)
df_dropped['Shipping and handling'] = df_dropped['Shipping and handling'].astype(float)


df_dropped['Gross transaction amount'] = df_dropped['Gross transaction amount'].replace('--', 0)
df_dropped['Gross transaction amount'] = df_dropped['Gross transaction amount'].fillna(0)
df_dropped['Gross transaction amount'] = df_dropped['Gross transaction amount'].astype(float)

df_dropped['eBay collected tax'] = df_dropped['eBay collected tax'].replace('--', 0)
df_dropped['eBay collected tax'] = df_dropped['eBay collected tax'].fillna(0)
df_dropped['eBay collected tax'] = df_dropped['eBay collected tax'].astype(float)



advertising = df_dropped[df_dropped['Description'].str.contains('Ad Fee Standard', na=False)]
df_dropped['AdvertisingFee_Ebay'] = advertising['Gross transaction amount'].copy()
df_dropped['AdvertisingFee_Ebay'] = df_dropped['AdvertisingFee_Ebay'].fillna(0)

df_dropped['ProductSales'] = df_dropped['Item subtotal'].cumsum(skipna=True)


df_dropped['Final Value Fee - fixed'] = df_dropped['Final Value Fee - fixed'].replace('--', 0)
df_dropped['Final Value Fee - fixed'] = df_dropped['Final Value Fee - fixed'].fillna(0)
df_dropped['Final Value Fee - fixed'] = df_dropped['Final Value Fee - fixed'].astype(float)
df_dropped['Final Value Fee - variable'] = df_dropped['Final Value Fee - variable'].replace('--', 0)
df_dropped['Final Value Fee - variable'] = df_dropped['Final Value Fee - variable'].fillna(0)
df_dropped['Final Value Fee - variable'] = df_dropped['Final Value Fee - variable'].astype(float)
df_dropped['EbayCharge'] = df_dropped['Final Value Fee - fixed'] + df_dropped['Final Value Fee - variable']
df_dropped.drop(['Final Value Fee - fixed', 'Final Value Fee - variable'], axis=1, inplace=True)

df_dropped['Payout_to_SSG603'] = df_dropped['Net amount'].cumsum(skipna=True)
df_dropped['Shipping_Income'] = df_dropped['Shipping and handling'].cumsum(skipna=True)
df_dropped['TaxWithheld_Ebay'] = df_dropped['eBay collected tax'].cumsum(skipna=True)
df_dropped['EbayFee'] = df_dropped['EbayCharge'].cumsum(skipna=True)
df_dropped['AdvertisingFee_Cummulative'] = df_dropped['AdvertisingFee_Ebay'].cumsum(skipna=True)


df_editted = df_dropped[['date', 'ProductSales', 'Shipping_Income', 'EbayFee', 'AdvertisingFee_Cummulative', 'Payout_to_SSG603']].copy()

df_final = df_editted.tail(1)
# df_final['CompareTotal'] =  df_final['ProductSales'] + df_final['Shipping_Income'] + df_final['EbayFee'] + df_final['AdvertisingFee_Cummulative']
df_final = df_final.set_index('date')
df_final = df_final.rename_axis('index')
df_final = df_final.T
df_final.reset_index(inplace=True)
df_final.rename(columns={0:'eBay'}, inplace=True)

df_final = df_final.round(2)
df_final

index,eBay,2021-12-19 00:00:00
0,ProductSales,131.14
1,Shipping_Income,38.00
2,EbayFee,-23.38
3,AdvertisingFee_Cummulative,-9.73
4,Payout_to_SSG603,136.03


In [4]:
df_dropped.tail()

,date,Net amount,Order number,Buyer name,Item title,Description,Item subtotal,eBay collected tax,Shipping and handling,Gross transaction amount,AdvertisingFee_Ebay,ProductSales,EbayCharge,Payout_to_SSG603,Shipping_Income,TaxWithheld_Ebay,EbayFee,AdvertisingFee_Cummulative
7,2021-12-09,9.09,05-07988-21827,Malissa White,Ball Pit WOO HOO Sugar Glider Toy,Not Ad Fee,4.53,0.27,6.25,10.78,0.00,63.32,-1.69,79.00,32.0,4.37,-13.03,-3.29
8,2021-12-15,16.16,16-08015-77045,Morgan Alexandra Willis,Bonding Pouch Jaguar for Sugar Gliders and sma...,Not Ad Fee,17.82,0.00,1.00,18.82,0.00,81.14,-2.66,95.16,33.0,4.37,-15.69,-3.29
9,2021-12-15,-1.69,--,--,--,Ad Fee Standard,0.00,0.00,0.00,-1.69,-1.69,81.14,0.00,93.47,33.0,4.37,-15.69,-4.98
10,2021-12-19,47.31,12-08031-38270,kenneth carter,Original Sugar Glider Bonding Potion - 2 oz. S...,Not Ad Fee,50.00,3.85,5.00,55.00,0.00,131.14,-7.69,140.78,38.0,8.22,-23.38,-4.98
11,2021-12-19,-4.75,--,--,--,Ad Fee Standard,0.00,0.00,0.00,-4.75,-4.75,131.14,0.00,136.03,38.0,8.22,-23.38,-9.73


In [4]:
# july = pd.ExcelWriter('excel_output/ebay_july2021.xlsx')

# # Write each dataframe to a different worksheet.
# df_dropped.to_excel(july, sheet_name='Sheet1')
# df_final.to_excel(july, sheet_name='Totals', index=False)


# # Close the Pandas Excel writer and output the Excel file.
# july.save()

In [5]:
# august = pd.ExcelWriter('excel_output/ebay_august2021.xlsx')

# # Write each dataframe to a different worksheet.
# df_dropped.to_excel(august, sheet_name='Sheet1')
# df_final.to_excel(august, sheet_name='Totals', index=False)


# # Close the Pandas Excel writer and output the Excel file.
# august.save()

In [6]:
# september = pd.ExcelWriter('excel_output/ebay_september2021.xlsx')

# # Write each dataframe to a different worksheet.
# df_dropped.to_excel(september, sheet_name='Sheet1')
# df_final.to_excel(september, sheet_name='Totals', index=False)


# # Close the Pandas Excel writer and output the Excel file.
# september.save()

In [5]:
december = pd.ExcelWriter('excel_output/ebay_december2021.xlsx')

# Write each dataframe to a different worksheet.
df_dropped.to_excel(december, sheet_name='Sheet1')
df_final.to_excel(december, sheet_name='Totals', index=False)


# Close the Pandas Excel writer and output the Excel file.
december.save()